This notebook:
1. Takes testing problems from /test/MATH.
1. Runs function "answer" on them, which **calls GPT-3.5-turbo**.
1. Saves the result in **results-baseline-turbo** under "gpt-answer".

This also has a modified version of the function that includes a "skip" functionality in case anything gets interrupted.

In [22]:
import os
import json
import openai

with open('keys.json') as f:
    data = json.load(f)
openai.api_key = data['openai-key']

In [23]:
# Test code block: run this to test the writing/reading of the modified data works, before you call GPT-3 API and potentially waste credits
def answer(problem):
    return "We answered " + problem

In [24]:
with open('prompts.json') as f:
    data = json.load(f)
prompt_json = data['no_wolfram']

# Calls GPT-3.5-Turbo API on input text & returns output.
def answer(problem):

    full_message = "[Problem] " + problem + "\n\n" + prompt_json
    
    # Use the OpenAI API to generate a response
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages = [{"role": "user", "content": full_message}],
        temperature=0, # temperature 0 keeps it deterministic
    )

    text_output = response['choices'][0]['message']['content']
    print(text_output)
    return text_output    

In [25]:
# Test code block: run this to test that davinci works, before you call GPT-3 API and potentially waste credits. Will print twice.
answer("What is 4+5?")

$\boxed{9}$


'$\\boxed{9}$'

In [28]:
# Define the paths to the data directories
data_dir = "test/MATH"
modified_data_dir = "results-baseline-turbo/MATH"

# Loop over the directories | possible: ["train", "test"]:
for data_type in ["test"]:
    # Loop over the subdirectories in the data directory
    for sub_dir in os.listdir(os.path.join(data_dir, data_type)):
        
        # Skip any files that are not directories
        if not os.path.isdir(os.path.join(data_dir, data_type, sub_dir)):
            continue
        
        # Create the corresponding subdirectory in the modified data directory
        os.makedirs(os.path.join(modified_data_dir, data_type, sub_dir), exist_ok=True)

        # Get a list of all the JSON files in the original subdirectory
        json_files = [f for f in os.listdir(os.path.join(data_dir, "test", sub_dir)) if f.endswith(".json")]
        
        # Loop over the selected JSON files in the original subdirectory
        for file_name in json_files:

            # This loop skips files that have already been modified.
            if not os.path.exists(os.path.join(modified_data_dir, data_type, sub_dir, file_name)):
            
                # Load the JSON file
                with open(os.path.join(data_dir, data_type, sub_dir, file_name), "r") as file:
                    data = json.load(file)
                
                # Extract the problem field
                problem = data["problem"]
                
                # Call the answer function
                answer_text = answer(problem)
                
                # Add the answer field to the JSON data
                data["gptanswer"] = answer_text
                
                # Save the modified JSON data to a new file
                with open(os.path.join(modified_data_dir, data_type, sub_dir, file_name), "w") as file:
                    json.dump(data, file)
            
            else:
                print(f"skipped {file_name}")

                # Load the JSON file
                with open(os.path.join(modified_data_dir, data_type, sub_dir, file_name), "r") as file:
                    data = json.load(file)
                
                problem = data["problem"]
                solution = data["solution"]
                gptanswer = data["gptanswer"]
                print(f"problem: {problem}")
                print(f"solution: {solution}")
                print(f"answer: {gptanswer}")

skipped 586.json
problem: What is $\dbinom{n}{n}$ for any positive integer $n$?
solution: $\dbinom{n}{n}=\dfrac{n!}{n!0!}=\boxed{1}$.  Also, there is only one way to choose $n$ objects out of $n$, which is simply choosing all of them.
answer: $$\dbinom{n}{n}=\boxed{1}$$
skipped 139.json
problem: How many paths are there from $A$ to $B$ on the lines of the grid shown, if every step must be up or to the right?[asy]size(3cm,3cm);int w=6;int h=3;int i;for (i=0; i<h; ++i){ draw((0,i) -- (w-1,i));}for (i=0; i<w; ++i){ draw((i, 0)--(i,h-1));}label("$B$", (w-1,h-1), NE);label("$A$", (0,0), SW);[/asy]
For example, here is one such valid path highlighted in blue:
[asy]
size(3cm,3cm);
int w=6;
int h=3;
int i;
for (i=0; i<h; ++i){ 
draw((0,i) -- (w-1,i));
}for (i=0; i<w; ++i){ 
draw((i, 0)--(i,h-1));
}
label("$B$", (w-1,h-1), NE);
label("$A$", (0,0), SW);

draw((0,0)--(1,0), blue+1.5);
draw((1,0)--(1,1), blue+1.5);
draw((1,1)--(2,1), blue+1.5);
draw((2,1)--(3,1), blue+1.5);
draw((3,1)--(4,1), blue